In [1]:
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

import pandas as pd
from math import pi
import datetime
import numpy as np

from collections import defaultdict

from bokeh.layouts import row, layout
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

cmc = CoinMarketCapAPI('7d386d4c-6c26-48f1-8335-8d255b23c4b4')

In [13]:
holds[holds['crypto'] == 'ADA']

,crypto,amount,price,value,timestamp
7,ADA,5446.153684,1.425859,7765.449737,2021-08-07 12:20:24.598516


In [2]:
holds = pd.DataFrame([['BTC', 1], 
                      ['ETH', 11], #Coinbase/CoinbaseWallet/CoinbasePro
                      ['GRT', 0],
                      ['AAVE', 0],
                      ['MIR', 130],
                      ['ERN', 0],
                      ['DOT', 400],
                      ['ADA', 10000], #From Kraken/Voyager/Coinbase
                      ['LINK', 100],
                      ['VGX', 800],
                      ['ONE', 4000],
                      ['BNB', 8], #Metamask/BinanceUS
                      ['AVAX', 30],
                     ],
                    columns = ['crypto', 'amount']
                    )

In [3]:
price = []
value = []

for crypto in holds['crypto']:
    try:
        data = cmc.cryptocurrency_quotes_latest(symbol=crypto).data
        price.append(data[crypto]['quote']['USD']['price'])
    except Exception as ex:
        print('error', '\n', ex)
holds['price'] = price

for index, row in holds.iterrows():
    value.append(row['amount'] * row['price'])
holds['value'] = value

time = datetime.datetime.now()
holds['timestamp'] = [time]*len(holds)

holds.to_csv('./crypto.csv', mode='a', header=False, index=False)

In [4]:
data     = pd.DataFrame(pd.read_csv('./crypto.csv'))
tempData = data[data['timestamp'] == data['timestamp'].max()]



###output_file("CryptoPortfolio.html")

#output_file("pie.html")
tempData['angle'] = tempData['value']/tempData['value'].sum() * 2*pi
tempData['color'] = Category20c[len(tempData)]
p1 = figure(plot_height=600, 
            title="Crypto Portfolio", 
            toolbar_location=None,
            tools="hover", 
            tooltips=[('Ticker', '@crypto'),
                      ('Value', '$@value{0.00}'),
                      ('Price($)', '$@price{0.00}'),
                      ('Amount', '@amount{0.00}')], 
            x_range=(-0.5, 1.0))
p1.wedge(x=0, 
         y=1, 
         radius=0.4,
         start_angle=cumsum('angle', include_zero=True), 
         end_angle=cumsum('angle'),
         line_color="white", 
         fill_color='color', 
         legend_field='crypto', 
         source=tempData)
p1.axis.axis_label=None
p1.axis.visible=False
p1.grid.grid_line_color = None



#output_file("colormapped_bars.html")
source = ColumnDataSource(data = dict(crypto = tempData['crypto'], 
                                      value = tempData['value'], color = tempData['color'], 
                                      price = tempData['price'],
                                      amount = tempData['amount']))
p2 = figure(x_range=tempData['crypto'], 
            y_range=(0,tempData['value'].max()),
            plot_height=600, 
            tools="hover", 
            tooltips=[('Ticker', '@crypto'),
                      ('Value', '$@value{0.00}'),
                      ('Price($)', '$@price{0.00}'),
                      ('Amount', '@amount{0.00}')],
            toolbar_location=None, 
            title='')
p2.vbar(x='crypto', 
        top='value', 
        width=.9, 
        color='color', 
        source=source, 
        fill_color = 'color')
p2.xgrid.grid_line_color = None



#output_file("sum.html")
source = ColumnDataSource(data = dict(crypto = ['sum'], 
                                      value = [tempData['value'].sum()], 
                                      color = [tempData['color'].tolist()[0]]))
p3 = figure(x_range=['sum'], 
            y_range=(0, 
            tempData['value'].sum()), 
            plot_height=600,
            tools="hover", 
            tooltips="@crypto: @value{0.00}", 
            toolbar_location=None, 
            title='')
p3.vbar(x='crypto', 
        top='value', 
        width=.1, 
        color='color', 
        source=source, 
        fill_color = 'color')
p3.xgrid.grid_line_color = None



#output_file('lines.html')
times     = data['timestamp'].value_counts().index.sort_values()
cryptos   = data['crypto'].value_counts().index.tolist()
color     = [tempData.loc[tempData[tempData.crypto == crypto].index.tolist(), 'color'].tolist()[0] for crypto in cryptos]
values    = [data[data['crypto'] == crypto]['value'].tolist() for crypto in cryptos]
values = [data[data['crypto'] == crypto]['value'].tolist() for crypto in cryptos]
nValues = []
for value in values:
    if len(value) < max(len(value) for value in values):
        temp = ([0]*(max(len(value) for value in values) - len(value)))
        value = temp + value
        nValues.append(value)
    else:
        nValues.append(value)
line = pd.DataFrame(data=np.transpose(nValues), 
                    columns = tuple(cryptos), 
                    index = pd.DatetimeIndex(times))
p4 = figure(width=600, 
            height=600,
            tools='hover',
            x_axis_type = 'datetime', 
            title = 'Portfolio History') 
p4.multi_line(xs = [line.index.values]*len(line.columns),
              ys = [line[crypto].values for crypto in line],
              line_color = tuple(color),
              line_width=1)


#output_file('percentlines.html')
percentline = pd.DataFrame(data=np.transpose(nValues), 
                           columns = tuple(cryptos), 
                           index = pd.DatetimeIndex(times))
for index in line.index:
    percentline.loc[index] = percentline.loc[index]/percentline.loc[index].sum()*100
p5 = figure(width=600, 
            height=600,
            tools='hover',
            x_axis_type = 'datetime') 
p5.multi_line(xs = [percentline.index.values]*len(percentline.columns),
              ys = [percentline[crypto].values for crypto in percentline],
              line_color = tuple(color),
              line_width=1)


sumline = pd.DataFrame(data = [row.sum() for index, row in line.iterrows()], 
                       index=[index for index, row in line.iterrows()], 
                       columns = ['sum'])
p6 = figure(width=600, 
            height=600,
            tools='hover',
            x_axis_type = 'datetime') 
p6.multi_line(xs = [sumline.index.values],
              ys = [sumline['sum'].values],
              line_color = tempData['color'].tolist()[0],
              line_width=1)

show(layout([[p1,p2,p3],[p4,p5,p6]]))

<ipython-input-4-1524e9939ef8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempData['angle'] = tempData['value']/tempData['value'].sum() * 2*pi
<ipython-input-4-1524e9939ef8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempData['color'] = Category20c[len(tempData)]


In [5]:
newline

NameError: name 'newline' is not defined

In [ ]:
from collections import defaultdict
import numpy as np
from scipy.stats import norm
from bokeh.palettes import Viridis6


In [ ]:
RT_x = np.linspace(118, 123, num=50)
mass_spec = defaultdict(list)
for scale, mz in [(1.0, 83), (0.9, 55), (0.6, 98), (0.4, 43), (0.2, 39), (0.12, 29)]:
    mass_spec["RT"].append(RT_x)
    mass_spec["RT_intensity"].append(norm(loc=120.4).pdf(RT_x) * scale)
    mass_spec['MZ_tip'].append(mz)
    mass_spec['Intensity_tip'].append(scale)
mass_spec['color'] = Viridis6

In [55]:
defaultdict(list)

defaultdict(list, {})

In [46]:
mass_spec

defaultdict(list,
            {'RT': [array([118.        , 118.10204082, 118.20408163, 118.30612245,
                     118.40816327, 118.51020408, 118.6122449 , 118.71428571,
                     118.81632653, 118.91836735, 119.02040816, 119.12244898,
                     119.2244898 , 119.32653061, 119.42857143, 119.53061224,
                     119.63265306, 119.73469388, 119.83673469, 119.93877551,
                     120.04081633, 120.14285714, 120.24489796, 120.34693878,
                     120.44897959, 120.55102041, 120.65306122, 120.75510204,
                     120.85714286, 120.95918367, 121.06122449, 121.16326531,
                     121.26530612, 121.36734694, 121.46938776, 121.57142857,
                     121.67346939, 121.7755102 , 121.87755102, 121.97959184,
                     122.08163265, 122.18367347, 122.28571429, 122.3877551 ,
                     122.48979592, 122.59183673, 122.69387755, 122.79591837,
                     122.89795918, 123.        ]),
 

In [44]:
pd.DataFrame(mass_spec)

,RT,RT_intensity,MZ_tip,Intensity_tip,color
0,"[118.0, 118.10204081632654, 118.20408163265306...","[0.022394530294842594, 0.028460254666943796, 0...",83,1.00,#440154
1,"[118.0, 118.10204081632654, 118.20408163265306...","[0.020155077265358336, 0.025614229200249417, 0...",55,0.90,#404387
2,"[118.0, 118.10204081632654, 118.20408163265306...","[0.013436718176905556, 0.017076152800166278, 0...",98,0.60,#29788E
3,"[118.0, 118.10204081632654, 118.20408163265306...","[0.008957812117937038, 0.011384101866777518, 0...",43,0.40,#22A784
4,"[118.0, 118.10204081632654, 118.20408163265306...","[0.004478906058968519, 0.005692050933388759, 0...",39,0.20,#79D151
5,"[118.0, 118.10204081632654, 118.20408163265306...","[0.002687343635381111, 0.0034152305600332554, ...",29,0.12,#FDE724


In [49]:
line = {}
line['values'] = nValues
line['cryptos'] = cryptos
line['color'] = tuple(color)
line['timestamp'] = times
line

{'values': [[4639.5298188833285,
   4642.344228121838,
   4634.433469813241,
   4658.401492712224,
   4657.435907614881,
   4638.225386648672,
   4647.3540296234005,
   4651.833713578438,
   4621.053501877328,
   4686.552833152099,
   4680.947950839486,
   4849.974040526537,
   4853.656455121959,
   4890.720700841856,
   4927.356098565158,
   4910.135634412506,
   4971.741737643496,
   5036.841944875287,
   5067.734825706674,
   5054.414517513084,
   5034.784599946297,
   5535.414690188217,
   5489.585732156855,
   5535.332788526874,
   5510.972303599273,
   5499.918438172718,
   5621.8492510667675,
   5648.725378952961,
   5644.08533099292,
   5680.774587602789,
   5684.626517357061,
   5640.278924064361,
   5686.161631045457,
   5638.258330471566,
   5619.556230290906,
   5601.950765582059,
   5630.8521641514235,
   5560.937193812164,
   5323.280173149134,
   5308.752619969764,
   4978.267519683806,
   4853.028338729812,
   4869.055221166429,
   4715.065048600051,
   4244.36284199865

In [47]:
nValues

[[4639.5298188833285,
  4642.344228121838,
  4634.433469813241,
  4658.401492712224,
  4657.435907614881,
  4638.225386648672,
  4647.3540296234005,
  4651.833713578438,
  4621.053501877328,
  4686.552833152099,
  4680.947950839486,
  4849.974040526537,
  4853.656455121959,
  4890.720700841856,
  4927.356098565158,
  4910.135634412506,
  4971.741737643496,
  5036.841944875287,
  5067.734825706674,
  5054.414517513084,
  5034.784599946297,
  5535.414690188217,
  5489.585732156855,
  5535.332788526874,
  5510.972303599273,
  5499.918438172718,
  5621.8492510667675,
  5648.725378952961,
  5644.08533099292,
  5680.774587602789,
  5684.626517357061,
  5640.278924064361,
  5686.161631045457,
  5638.258330471566,
  5619.556230290906,
  5601.950765582059,
  5630.8521641514235,
  5560.937193812164,
  5323.280173149134,
  5308.752619969764,
  4978.267519683806,
  4853.028338729812,
  4869.055221166429,
  4715.065048600051,
  4244.362841998652,
  4267.828332565327,
  4708.5365195070035,
  5051.66

In [45]:
line = pd.DataFrame(data=np.transpose(nValues), 
                    columns = tuple(cryptos), 
                    index = pd.DatetimeIndex(times))
line = line.reset_index().rename(columns={'index':'timestamp'})
line

,timestamp,ADA,ONE,VGX,BTC,CAKE,DOT,AAVE,MIR,BUNNY,LINK,GRT,ETH,BNB,ERN,AVAX
0,2021-04-08 10:22:25.878621,4639.529819,405.999714,3674.982625,8969.346867,3618.251759,10218.592730,852.175859,1245.600930,610.272610,3090.075246,5408.364504,20617.841476,3007.443001,357.220200,0.000000
1,2021-04-08 11:11:20.810459,4642.344228,400.713650,3634.986644,8978.112032,3578.612919,10224.356418,850.950577,1243.430460,607.797768,3107.536035,5357.234238,20634.687964,3029.750757,364.977434,0.000000
2,2021-04-08 11:32:46.040530,4634.433470,395.319267,3591.700858,8966.875529,3659.350072,10215.743409,854.166843,1244.577696,610.657852,3107.271229,5327.702795,20617.331752,3035.603420,364.455229,0.000000
3,2021-04-08 12:35:44.395035,4658.401493,397.466555,3615.069137,8975.674266,3598.315986,10301.877622,861.314799,1246.345674,636.795958,3129.473985,5347.214273,20899.482971,3026.588374,370.868236,0.000000
4,2021-04-08 15:39:58.477644,4657.435908,388.088179,3657.694536,9032.833644,3636.515696,10347.643448,873.358091,1249.631376,637.218266,3093.147302,5347.095553,20911.241793,3042.409449,334.255847,0.000000
5,2021-04-08 16:13:12.826036,4638.225387,391.450572,3662.823463,8984.890573,3650.236472,10320.664123,854.661393,1237.710686,638.590429,3097.384440,5367.218224,20837.532449,3050.287221,355.890192,0.000000
6,2021-04-09 09:37:58.737579,4647.354030,410.190004,3544.713678,9126.355183,3893.825319,10379.004986,859.281831,1467.069165,712.513582,3093.216687,5586.104626,20993.848232,3253.374840,331.676859,0.000000
7,2021-04-09 09:43:36.208091,4651.833714,406.213781,3545.658814,9135.179191,3952.184566,10403.914109,859.708078,1464.269542,715.284813,3094.122407,5604.034541,21004.944714,3122.072209,331.292397,0.000000
8,2021-04-09 18:01:28.683019,4621.053502,396.713375,3582.491711,9071.660565,4211.612420,10278.191551,849.174175,1616.152879,741.022557,3049.979728,5416.756388,20928.588259,3096.291738,303.421702,0.000000
9,2021-04-10 09:38:10.478999,4686.552833,373.961334,3578.927906,9353.971296,3924.757048,10425.634214,865.549046,1621.868779,786.841530,3081.524099,5727.380340,21666.627825,3313.339789,303.502641,0.000000


In [185]:
percentline = pd.DataFrame(data=np.transpose(values), columns = tuple(cryptos), index = pd.DatetimeIndex(times))
[percentline.loc[index] = percentline.loc[index]/sum_*100 for index in line.index]

In [9]:
for index in line.index:
    print(percentline.loc[index].sum())
    percentline.loc[index] = percentline.loc[index]/sum_*100
percentline

66715.69733929433


NameError: name 'sum_' is not defined